In [49]:
from interpretable_ssl.immune.trainer import ImmuneTrainer

trainer = ImmuneTrainer()
train_loader, test_loader = trainer.get_train_test_loader()

loading data
training with number of prototypes : 32
train dataset size:  29137


In [50]:
import torch
from sklearn.metrics import classification_report

def report(trainer):
    model = trainer.get_model()
    model_path = trainer.get_model_path()
    model.load_state_dict(torch.load(model_path)["model_state_dict"])
    model.to(trainer.device)
    train_loader, test_loader = trainer.get_train_test_loader()

    def to_flat_np_arr(tensor_list):
        tensor_list = [tensor.cpu().numpy() for tensor in tensor_list]
        flat_arr = np.concatenate(tensor_list).ravel()
        return flat_arr
        
    def calc_outputs(loader):
        total_acc = 0
        y_pred, y_true = [], []
        for x, y in loader:
            z, decoded, logits = model(x)
            preds = logits.argmax(dim=1)
            acc = (preds == y).sum() / len(y)
            y_pred.append(preds)
            y_true.append(y)
            total_acc += acc
        print('acc',total_acc / len(loader))
        y_pred = to_flat_np_arr(y_pred)
        y_true = to_flat_np_arr(y_true)
        return y_true, y_pred
    
    def print_report(loader):
        y_true, y_pred = calc_outputs(loader)
        y_true = trainer.dataset.le.inverse_transform(y_true)
        y_pred = trainer.dataset.le.inverse_transform(y_pred)
        print(classification_report(y_true, y_pred))
    print('train')
    print_report(train_loader)
    print('test')
    print_report(test_loader)


In [ ]:
report(trainer)